In [57]:
import json
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

In [58]:
def load_data():
    with open('data/song.json') as inf:
        data = json.load(inf)
    data = pd.DataFrame(data)
    data.set_index('id', inplace=True)
    data['time_played'] = pd.to_datetime(data.time_played)
    data['user_sign_up_date'] = pd.to_datetime(data.user_sign_up_date)
    
    return data

In [59]:
data = load_data()

In [9]:
# сгруппируем по юзерам из различных штатов

In [60]:
user_counts = data.groupby('user_state').user_id.agg(lambda ids: len(np.unique(ids)))

In [61]:
user_counts

user_state
Alabama            4
Alaska             2
Arizona            1
Arkansas           2
California        21
Colorado           3
Connecticut        1
Florida            7
Georgia            6
Idaho              1
Illinois           7
Indiana            4
Iowa               1
Kansas             1
Kentucky           3
Louisiana          5
Maryland           5
Massachusetts      6
Michigan           5
Minnesota          4
Mississippi        3
Missouri           5
Nebraska           1
New Jersey         6
New Mexico         1
New York          23
North Carolina     6
North Dakota       1
Ohio               9
Oklahoma           2
Oregon             3
Pennsylvania       9
Rhode Island       1
South Carolina     3
Tennessee          5
Texas             15
Utah               2
Virginia           2
Washington         2
West Virginia      3
Wisconsin          5
Name: user_id, dtype: int64

In [62]:
user_counts.sort_values(ascending=False,inplace=True)

In [63]:
user_counts[:3]

user_state
New York      23
California    21
Texas         15
Name: user_id, dtype: int64

In [64]:
#Я определяю «среднее игровое событие в час» как метрику для измерения вовлеченности пользователей в состояние.

In [65]:
data.shape[0]

4000

In [66]:
def count_by_state(df):
    total_played = df.shape[0]
    first_play_dt = df.time_played.min()
    last_play_dt = df.time_played.max()
    duration = last_play_dt - first_play_dt
    duration_hours = duration.total_seconds()/3600
    return pd.Series([first_play_dt, last_play_dt, duration, duration_hours, total_played], 
                    index=['first_play_dt', 'last_play_dt', 'duration', 'duration_hours', 'total_played'])

In [67]:
count_by_state(data)

first_play_dt     2015-06-01 05:02:54
last_play_dt      2015-06-28 23:46:06
duration             27 days 18:43:12
duration_hours                 666.72
total_played                     4000
dtype: object

In [68]:
# теперь применим к каждому юзеру т.е. сгруппируем по штату пользователя
counts_by_states = data.groupby('user_state').apply(count_by_state)

In [69]:
counts_by_states

,first_play_dt,last_play_dt,duration,duration_hours,total_played
user_state,,,,,
Alabama,2015-06-01 14:17:56,2015-06-28 14:48:55,27 days 00:30:59,648.516389,104
Alaska,2015-06-01 18:48:18,2015-06-28 22:58:23,27 days 04:10:05,652.168056,58
Arizona,2015-06-01 13:05:17,2015-06-28 19:23:45,27 days 06:18:28,654.307778,22
Arkansas,2015-06-01 14:07:37,2015-06-28 17:40:11,27 days 03:32:34,651.542778,34
California,2015-06-01 06:33:03,2015-06-28 20:35:50,27 days 14:02:47,662.046389,425
Colorado,2015-06-01 14:57:48,2015-06-28 11:16:31,26 days 20:18:43,644.311944,54
Connecticut,2015-06-06 19:23:58,2015-06-28 13:16:32,21 days 17:52:34,521.876111,16
Florida,2015-06-01 09:29:39,2015-06-28 22:59:27,27 days 13:29:48,661.496667,180
Georgia,2015-06-01 06:41:36,2015-06-28 21:37:34,27 days 14:55:58,662.932778,135


In [70]:
counts_by_states['hr_average'] = counts_by_states.total_played/counts_by_states.duration_hours

In [71]:
counts_by_states.sort_values(by='hr_average', ascending=False, inplace=True)

In [72]:
counts_by_states.head()

,first_play_dt,last_play_dt,duration,duration_hours,total_played,hr_average
user_state,,,,,,
New York,2015-06-01 06:14:45,2015-06-28 21:36:40,27 days 15:21:55,663.365278,469,0.707001
California,2015-06-01 06:33:03,2015-06-28 20:35:50,27 days 14:02:47,662.046389,425,0.641949
Texas,2015-06-01 06:09:04,2015-06-28 20:28:35,27 days 14:19:31,662.325278,230,0.347261
Ohio,2015-06-01 05:02:54,2015-06-28 22:22:25,27 days 17:19:31,665.325278,209,0.314132
Florida,2015-06-01 09:29:39,2015-06-28 22:59:27,27 days 13:29:48,661.496667,180,0.272110


In [73]:
counts_by_states.tail()

,first_play_dt,last_play_dt,duration,duration_hours,total_played,hr_average
user_state,,,,,,
Iowa,2015-06-01 15:43:51,2015-06-28 03:34:29,26 days 11:50:38,635.843889,23,0.036172
Arizona,2015-06-01 13:05:17,2015-06-28 19:23:45,27 days 06:18:28,654.307778,22,0.033623
Connecticut,2015-06-06 19:23:58,2015-06-28 13:16:32,21 days 17:52:34,521.876111,16,0.030659
New Mexico,2015-06-01 05:22:30,2015-06-28 13:15:58,27 days 07:53:28,655.891111,17,0.025919
Kansas,2015-06-05 15:01:50,2015-06-27 09:02:15,21 days 18:00:25,522.006944,8,0.015325


In [74]:
user_sign = data.groupby('user_state')[['user_id','user_sign_up_date']].agg({'user_sign_up_date':'min'}).reset_index()

In [75]:
data

,user_id,user_state,user_sign_up_date,song_played,time_played
id,,,,,
GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00
...,...,...,...,...,...
MKVEXMRJGF,122,Louisiana,2015-05-16,Paperback Writer,2015-06-03 08:26:26
HMBEUWPBQI,76,New York,2015-05-08,Reprise / Day in the Life,2015-06-14 15:46:04
KIYDZUFUJN,68,Florida,2015-05-08,Get Back,2015-06-13 13:13:11


In [76]:
user_sign

,user_state,user_sign_up_date
0,Alabama,2015-05-01
1,Alaska,2015-05-12
2,Arizona,2015-05-12
3,Arkansas,2015-05-08
4,California,2015-05-04
5,Colorado,2015-05-19
6,Connecticut,2015-05-16
7,Florida,2015-05-04
8,Georgia,2015-05-02
9,Idaho,2015-05-19


In [77]:
pd.DataFrame(user_sign.values,columns=['user_state','user_id','sign_date']).sort_values(by='sign_date').head()

ValueError: Shape of passed values is (41, 2), indices imply (41, 3)